In [1]:
# Imports
import fasttext
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
# @title Figure Settings
import ipywidgets as widgets
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/content-creation/main/nma.mplstyle")

In [3]:
# @title Helper functions
def cosine_similarity(vec_a, vec_b):
  """Compute cosine similarity between vec_a and vec_b"""
  return np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))


def getSimilarity(word1, word2):
  v1 = ft_en_vectors.get_word_vector(word1)
  v2 = ft_en_vectors.get_word_vector(word2)
  return cosine_similarity(v1, v2)

In [4]:
# @title Set random seed

# @markdown Executing `set_seed(seed=seed)` you are setting the seed

# For DL its critical to set the random seed so that students can have a
# baseline to compare their results to expected results.
# Read more here: https://pytorch.org/docs/stable/notes/randomness.html

# Call `set_seed` function in the exercises to ensure reproducibility.
import random
import torch

def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness.
  NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')

# In case that `DataLoader` is used
def seed_worker(worker_id):
  """
  DataLoader will reseed workers following randomness in
  multi-process data loading algorithm.

  Args:
    worker_id: integer
      ID of subprocess to seed. 0 means that
      the data will be loaded in the main process
      Refer: https://pytorch.org/docs/stable/data.html#data-loading-randomness for more details

  Returns:
    Nothing
  """
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

In [5]:
# @title Set device (GPU or CPU). Execute `set_device()`

# Inform the user if the notebook uses GPU or CPU.

def set_device():
  """
  Set the device. CUDA if available, CPU otherwise

  Args:
    None

  Returns:
    Nothing
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device

In [6]:
DEVICE = set_device()
SEED = 2021
set_seed(seed=SEED)

Random seed 2021 has been set.


In [ ]:
import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore')  # English
ft_en_vectors = fasttext.load_model('/kaggle/input/fasttext-pretrained-crawl-vector-en-bin/cc.en.300.bin')

In [ ]:
import fasttext.util
# fasttext.util.download_model('es', if_exists='ignore')  # English
ft_es_vectors = fasttext.load_model('/kaggle/input/fasttext-spanish-300/cc.es.300.bin')

Lets see similarity without making bilingual embedding

In [ ]:
hello = ft_en_vectors.get_word_vector('hello')
hi = ft_en_vectors.get_word_vector('hi')
bonjour = ft_es_vectors.get_word_vector('bonjour')

print(f"Cosine Similarity between HI and HELLO: {cosine_similarity(hello, hi)}")
print(f"Cosine Similarity between BONJOUR and HELLO: {cosine_similarity(hello, bonjour)}")

In [ ]:
cat = ft_en_vectors.get_word_vector('cat')
chatte = ft_es_vectors.get_word_vector('chatte')
chat = ft_es_vectors.get_word_vector('chat')

print(f"Cosine Similarity between cat and chatte: {cosine_similarity(cat, chatte)}")
print(f"Cosine Similarity between cat and chat: {cosine_similarity(cat, chat)}")
print(f"Cosine Similarity between chatte and chat: {cosine_similarity(chatte, chat)}")

In [ ]:
en_words = set(ft_en_vectors.words)
es_words = set(ft_es_vectors.words)
overlap = list(en_words & es_words)
bilingual_dictionary = [(entry, entry) for entry in overlap]

In [ ]:
print(bilingual_dictionary)

In [ ]:
def make_training_matrices(source_dictionary, target_dictionary,
                           bilingual_dictionary):
  source_matrix = []
  target_matrix = []
  for (source, target) in tqdm(bilingual_dictionary):
    # if source in source_dictionary.words and target in target_dictionary.words:
    source_matrix.append(source_dictionary.get_word_vector(source))
    target_matrix.append(target_dictionary.get_word_vector(target))
  # return training matrices
  return np.array(source_matrix), np.array(target_matrix)


# from https://stackoverflow.com/questions/21030391/how-to-normalize-array-numpy
def normalized(a, axis=-1, order=2):
  """Utility function to normalize the rows of a numpy array."""
  l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
  l2[l2==0] = 1
  return a / np.expand_dims(l2, axis)


def learn_transformation(source_matrix, target_matrix, normalize_vectors=True):
  """
  Source and target matrices are numpy arrays, shape
  (dictionary_length, embedding_dimension). These contain paired
  word vectors from the bilingual dictionary.
  """
  # optionally normalize the training vectors
  if normalize_vectors:
    source_matrix = normalized(source_matrix)
    target_matrix = normalized(target_matrix)
  # perform the SVD
  product = np.matmul(source_matrix.transpose(), target_matrix)
  U, s, V = np.linalg.svd(product)
  # return orthogonal transformation which aligns source language to the target
  return np.matmul(U, V)

In [ ]:
source_training_matrix, target_training_matrix = make_training_matrices(ft_en_vectors, ft_es_vectors, bilingual_dictionary)

In [ ]:
transform = learn_transformation(source_training_matrix, target_training_matrix)

Let’s run the same examples as above, but this time, whenever we use French words, the matrix multiplies the embedding by the transpose of the transform matrix. That works a lot better!

In [ ]:
hello = ft_en_vectors.get_word_vector('hello')
hi = ft_en_vectors.get_word_vector('hi')
bonjour = np.matmul(ft_es_vectors.get_word_vector('bonjour'), transform.T)

print(f"Cosine Similarity between HI and HELLO: {cosine_similarity(hello, hi)}")
print(f"Cosine Similarity between BONJOUR and HELLO: {cosine_similarity(hello, bonjour)}")

In [ ]:
cat = ft_en_vectors.get_word_vector('cat')
chatte = np.matmul(ft_es_vectors.get_word_vector('chatte'), transform.T)
chat = np.matmul(ft_es_vectors.get_word_vector('chat'), transform.T)

print(f"Cosine Similarity between cat and chatte: {cosine_similarity(cat, chatte)}")
print(f"Cosine Similarity between cat and chat: {cosine_similarity(cat, chat)}")
print(f"Cosine Similarity between chatte and chat: {cosine_similarity(chatte, chat)}")

Embeddings have now being obtained

In [ ]:
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from gensim.models import KeyedVectors
import nltk
# nltk.download()
import nltk
import pandas as pd

In [ ]:
# English
train_df_en = pd.read_csv('/kaggle/input/cleaned-data-eng/cleaned_train_df.csv')
val_df_en = pd.read_csv('/kaggle/input/cleaned-data-eng/cleaned_val_df.csv')
test_df_en = pd.read_csv('/kaggle/input/cleaned-data-eng/cleaned_test_df.csv')

# Spanish
train_df_es = pd.read_csv('/kaggle/input/aaa1111/STS-train/data/stsb-multi-mt-main/data/stsb-es-train.csv', usecols=[0,1,2], names=['sent1', 'sent2', 'score'], header=None)
val_df_es = pd.read_csv('/kaggle/input/aaa1111/STS-train/data/stsb-multi-mt-main/data/stsb-es-dev.csv', usecols=[0,1,2], names=['sent1', 'sent2', 'score'], header=None)
test_df_es = pd.read_csv('/kaggle/input/aaa1111/STS-train/data/stsb-multi-mt-main/data/stsb-es-test.csv', usecols=[0,1,2], names=['sent1', 'sent2', 'score'], header=None)

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.linear_model import LinearRegression
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [ ]:
def preprocess_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Replace numbers with num
    text = re.sub(r'\d+', '', text)
    # Lower case
    text= text.lower()
    sent_token = text.split()
    # Lemmatize
#     sent_token = [lemmatizer.lemmatize(word) for word in sent_token]
    # Stemming
#     sent_token = [ps.stem(word) for word in sent_token]
    return sent_token

In [ ]:
# train_df_en['sent1'] = train_df_en['sent1'].apply(lambda x: preprocess_text(x))
# train_df_en['sent2'] = train_df_en['sent2'].apply(lambda x: preprocess_text(x))

train_df_es['sent1'] = train_df_es['sent1'].apply(lambda x: preprocess_text(x))
train_df_es['sent2'] = train_df_es['sent2'].apply(lambda x: preprocess_text(x))

# val_df_en['sent1'] = val_df_en['sent1'].apply(lambda x: preprocess_text(x))
# val_df_en['sent2'] = val_df_en['sent2'].apply(lambda x: preprocess_text(x))

val_df_es['sent1'] = val_df_es['sent1'].apply(lambda x: preprocess_text(x))
val_df_es['sent2'] = val_df_es['sent2'].apply(lambda x: preprocess_text(x))

# test_df_en['sent1'] = test_df_en['sent1'].apply(lambda x: preprocess_text(x))
# test_df_en['sent2'] = test_df_en['sent2'].apply(lambda x: preprocess_text(x))

test_df_es['sent1'] = test_df_es['sent1'].apply(lambda x: preprocess_text(x))
test_df_es['sent2'] = test_df_es['sent2'].apply(lambda x: preprocess_text(x))

In [ ]:
total_sent_en = list(train_df_en['sent1']) + list(train_df_en['sent2'])
total_sent_es= list(train_df_es['sent1']) + list(train_df_es['sent2'])

In [ ]:
word_dict_en = {}
for word_tokens in total_sent_en:
    for word in word_tokens:
        if word in word_dict_en:
            word_dict_en[word] += 1
        else:
            word_dict_en[word] = 1
            
vocab_length_en = len(word_dict_en)

word_dict_es = {}
for word_tokens in total_sent_es:
    for word in word_tokens:
        if word in word_dict_es:
            word_dict_es[word] += 1
        else:
            word_dict_es[word] = 1
            
vocab_length_es = len(word_dict_es)

In [ ]:
def get_sorted_counter(word_counter):
    return {k: v for k, v in sorted(word_counter.items(), key=lambda item: item[1], reverse=False)}

In [ ]:
sorted_counter_en = get_sorted_counter(word_dict_en)
sorted_counter_es = get_sorted_counter(word_dict_es)

In [ ]:
train_df_en['sent1'] = train_df_en['sent1'].astype(str).apply(eval)
train_df_en['sent2'] = train_df_en['sent2'].astype(str).apply(eval)

train_df_es['sent1'] = train_df_es['sent1'].astype(str).apply(eval)
train_df_es['sent2'] = train_df_es['sent2'].astype(str).apply(eval)

val_df_en['sent1'] = val_df_en['sent1'].astype(str).apply(eval)
val_df_en['sent2'] = val_df_en['sent2'].astype(str).apply(eval)

val_df_es['sent1'] = val_df_es['sent1'].astype(str).apply(eval)
val_df_es['sent2'] = val_df_es['sent2'].astype(str).apply(eval)

test_df_en['sent1'] = test_df_en['sent1'].astype(str).apply(eval)
test_df_en['sent2'] = test_df_en['sent2'].astype(str).apply(eval)

test_df_es['sent1'] = test_df_es['sent1'].astype(str).apply(eval)
test_df_es['sent2'] = test_df_es['sent2'].astype(str).apply(eval)

In [ ]:
import pandas as pd

# assuming train_df_en and train_df_es are already defined and loaded with data

# selecting the "sent1" column from train_df_en
sent1_en = train_df_en["sent1"] 

# selecting the "sent2" column from train_df_es
sent2_es = train_df_es["sent2"] 

score = train_df_en["score"] 

# concatenating the selected columns along with the "score" column
new_df_one = pd.concat([sent1_en, sent2_es, score], axis=1)


# selecting the "sent1" column from train_df_en
sent1_en = train_df_en["sent2"] 

# selecting the "sent2" column from train_df_es
sent2_es = train_df_es["sent1"] 

score = train_df_en["score"] 

# concatenating the selected columns along with the "score" column
new_df_two = pd.concat([sent1_en, sent2_es, score], axis=1)

final_train = pd.concat([new_df_one, new_df_two], ignore_index=True)
# printing the new dataframe
# final_train.len()

In [ ]:
import pandas as pd

# assuming train_df_en and train_df_es are already defined and loaded with data

# selecting the "sent1" column from train_df_en
sent1_en = val_df_en["sent1"] 

# selecting the "sent2" column from train_df_es
sent2_es = val_df_es["sent2"] 

score = val_df_en["score"] 

# concatenating the selected columns along with the "score" column
new_df_one = pd.concat([sent1_en, sent2_es, score], axis=1)


# selecting the "sent1" column from train_df_en
sent1_en = val_df_es["sent2"] 

# selecting the "sent2" column from train_df_es
sent2_es = val_df_es["sent1"] 

score = val_df_en["score"] 

# concatenating the selected columns along with the "score" column
new_df_two = pd.concat([sent1_en, sent2_es, score], axis=1)

final_val = pd.concat([new_df_one, new_df_two], ignore_index=True)
# printing the new dataframe
# final_train.len()

In [ ]:
import pandas as pd

# assuming train_df_en and train_df_es are already defined and loaded with data

# selecting the "sent1" column from train_df_en
sent1_en = test_df_en["sent1"] 

# selecting the "sent2" column from train_df_es
sent2_es = test_df_es["sent2"] 

score = test_df_en["score"] 

# concatenating the selected columns along with the "score" column
new_df_one = pd.concat([sent1_en, sent2_es, score], axis=1)


# selecting the "sent1" column from train_df_en
sent1_en = test_df_es["sent2"] 

# selecting the "sent2" column from train_df_es
sent2_es = test_df_es["sent1"] 

score = test_df_en["score"] 

# concatenating the selected columns along with the "score" column
new_df_two = pd.concat([sent1_en, sent2_es, score], axis=1)

final_test = pd.concat([new_df_one, new_df_two], ignore_index=True)
# printing the new dataframe
# final_train.len()

In [ ]:
train_sentences1 = list(final_train['sent1'])
train_sentences2 = list(final_train['sent2'])
train_similarity_scores = list(final_train['score'])

val_sentences1 = list(final_val['sent1'])
val_sentences2 = list(final_val['sent2'])
val_similarity_scores = list(final_val['score'])

test_sentences1 = list(final_test['sent1'])
test_sentences2 = list(final_test['sent2'])
test_similarity_scores = list(final_test['score'])

In [ ]:
print(len(train_sentences1))

In [ ]:
def get_sentence_embedding_en(sentence, max_length=30):
        words = sentence
        # filter out words that are not present in the model's vocabulary
        words = [word for word in words if word in ft_en_vectors.words ]
        # generate word embeddings for each word
        embeddings = [ft_en_vectors.get_word_vector(word) for word in words]
        #mean strategy
        embedding = np.mean(embeddings, axis=0)
        return embedding
    
def get_sentence_embedding_es(sentence, max_length=30):
        words = sentence
        # filter out words that are not present in the model's vocabulary
        words = [word for word in words if word in ft_es_vectors.words ]
        # generate word embeddings for each word
        embeddings = [np.matmul(ft_es_vectors.get_word_vector(word), transform.T) for word in words]
        #mean strategy
        embedding = np.mean(embeddings, axis=0)
        return embedding

In [ ]:
# print( val_sentences2)

In [ ]:
# # generate sentence embeddings
# train_X1 = np.array([get_sentence_embedding_en(sentence) for sentence in train_sentences1])
# print("1")
# train_X2 = np.array([get_sentence_embedding_es(sentence) for sentence in train_sentences2])
# print("1")

# val_X1 = np.array([get_sentence_embedding_en(sentence) for sentence in val_sentences1])
# print("1")
# val_X2 = np.array([get_sentence_embedding_es(sentence) for sentence in val_sentences2])
# print("1")


# test_X1 = np.array([get_sentence_embedding_en(sentence) for sentence in test_sentences1])
# print("1")

# test_X2 = np.array([get_sentence_embedding_es(sentence) for sentence in test_sentences2])

In [ ]:
# print(train_X2)

In [ ]:
# train_X = np.concatenate([train_X1, train_X2], axis=1)
# val_X = np.concatenate([val_X1, val_X2], axis=1)
# test_X = np.concatenate([test_X1, test_X2], axis=1)

In [ ]:
def sts_score(sim_score):
    sts_score = (sim_score+1) * 2.5
    return sts_score

In [ ]:
from scipy import spatial
from scipy.stats import pearsonr

def get_sts_scores(emb1_lt, emb2_lt):
    y_pred = []
    for i in range(len(emb1_lt)):
        sim_score = 1 - spatial.distance.cosine(emb1_lt[i], emb2_lt[i])
        y_pred.append(sts_score(sim_score))
    return y_pred
    
def pearson_corr(y_true, y_pred):
    """
    Calculate Pearson correlation coefficient between two arrays.
    """
    corr, _ = pearsonr(y_true, y_pred)
    return corr

# Normalised Cosine Similarity

In [ ]:
import numpy as np
import torch
import fasttext.util
from torch import nn
from torch.utils.data import DataLoader

# Load FastText embeddings
ft_es_vectors = fasttext.load_model('/kaggle/input/fasttext-spanish-300/cc.es.300.bin')
ft_en_vectors = fasttext.load_model('/kaggle/input/fasttext-pretrained-crawl-vector-en-bin/cc.en.300.bin')

# Get the word vectors and corresponding words for both languages
words_es = ft_es_vectors.get_words()
vectors_es = np.array([ft_es_vectors.get_word_vector(word) for word in words_es])

words_en = ft_en_vectors.get_words()
vectors_en = np.array([ft_en_vectors.get_word_vector(word) for word in words_en])

# Convert the word vectors to PyTorch tensors
vectors_es_tensor = torch.from_numpy(vectors_es).float()
vectors_en_tensor = torch.from_numpy(vectors_en).float()

# Define a simple feed-forward neural network for mapping
class MappingNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MappingNetwork, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Create the mapping network
input_dim = vectors_es_tensor.shape[1]
output_dim = vectors_en_tensor.shape[1]
mapping_network = MappingNetwork(input_dim, output_dim)

# Define the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(mapping_network.parameters(), lr=0.001)

# Train the mapping network
dataset = DataLoader(list(zip(vectors_es_tensor, vectors_en_tensor)), batch_size=32, shuffle=True)

num_epochs = 10
total_steps = len(dataset) * num_epochs

for epoch in range(num_epochs):
    for step, batch in enumerate(dataset):
        src_vectors, tgt_vectors = batch
        optimizer.zero_grad()
        mapped_vectors = mapping_network(src_vectors)
        loss = loss_fn(mapped_vectors, tgt_vectors)
        loss.backward()
        optimizer.step()

        # Calculate progress in percentage
        progress = (epoch * len(dataset) + step + 1) / total_steps * 100

        # Print epoch and progress
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{len(dataset)}], Progress: {progress:.2f}%")

# Map the source language word vectors to the target language space
mapped_vectors_es = mapping_network(vectors_es_tensor).detach().numpy()

# Calculate cosine similarity between mapped source vectors and target vectors
cos_sim_scores = np.dot(mapped_vectors_es, vectors_en.T) / (
        np.linalg.norm(mapped_vectors_es, axis=1).reshape(-1, 1) * np.linalg.norm(vectors_en, axis=1))

# 


In [ ]:
Example of computing similarity between two words
word_idx_es = list(words_es).index('manzana')
word_idx_en = list(words_en).index('apple')
similarity_score = cos_sim_scores[word_idx_es][word_idx_en]
print("Similarity score between 'manzana' and 'apple':", similarity_score)

In [ ]:
train_y_pred = get_sts_scores(train_X1, train_X2)
val_y_pred = get_sts_scores(val_X1, val_X2)
test_y_pred = get_sts_scores(test_X1, test_X2)

In [ ]:
corr = pearson_corr(train_similarity_scores, train_y_pred)
print("Pearson correlation coefficient: {:.2f}".format(corr))

In [ ]:
corr = pearson_corr(val_similarity_scores, val_y_pred)
print("Pearson correlation coefficient: {:.2f}".format(corr))

In [ ]:
corr = pearson_corr(test_similarity_scores, test_y_pred)
print("Pearson correlation coefficient: {:.2f}".format(corr))

# Residual Analysis

In [ ]:
row_max_len = lambda row: max(len(row['sent1']), len(row['sent2']))

In [ ]:
train_df['sent_len'] = train_df.apply(row_max_len, axis=1)
val_df['sent_len'] = val_df.apply(row_max_len, axis=1)
test_df['sent_len'] = test_df.apply(row_max_len, axis=1)

In [ ]:
train_df['diff'] = abs(train_df['score'] - train_y_pred)
val_df['diff'] = abs(val_df['score'] - val_y_pred)
test_df['diff'] = abs(test_df['score'] - test_y_pred)

In [ ]:
grouped_df = train_df.groupby(['sent_len']).mean()


In [ ]:
grouped_df['sent_length'] = grouped_df.index

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(list(grouped_df.index), list(grouped_df['diff']))
plt.title("average error trend in training data")
plt.xlabel("sentence length")
plt.ylabel("average difference")
plt.show()

In [ ]:
grouped_val = val_df.groupby(['sent_len']).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(list(grouped_val.index), list(grouped_val['diff']))
plt.title("average error trend in validation data")
plt.xlabel("sentence length")
plt.ylabel("average difference")
plt.show()

In [ ]:
grouped_test = test_df.groupby(['sent_len']).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(list(grouped_test.index), list(grouped_test['diff']))
plt.title("average error trend in test data")
plt.xlabel("sentence length")
plt.ylabel("average difference")
plt.show()